In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Preperation and Cleaning


In [ ]:
data = pd.read_csv("../input/customer-personality-analysis/marketing_campaign.csv", sep='\t', index_col= 'ID')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

seems as if our missing values are only in the Income column. This could correspond to customers with no income so imputation would not make sense here. This dataset is large enough where we can omit these rows.



In [ ]:
data = data.dropna()
data.head()

Next we 
want to get a better idea of the ages of our customers. We do not currently have an Age column in our dataframe so lets create one using the Year_Birth feature.



In [ ]:
from datetime import date
#age = map(lambda x: date.today().year - x, data['Year_Birth'])
#data['Age'] = data.Year_birth.map(lambda x = date.today().year - data.Year_birth)

def get_age(birthyear):
    return date.today().year - birthyear

# ages = data.Year_Birth.map(get_age)

data["Age"] = data.Year_Birth.map(get_age)

data.Age.describe()

it seems that some users are extremely old, the oldest customer is 128 years old! This data might be incorrect. Lets take a closer look.



In [ ]:
data.sort_values('Year_Birth')

We observe that we have three customers who were born in the 19th century. Surely this cannot be correct. Lets drop these rows.



In [ ]:
#data[data.Year_Birth < 1900].drop(axis = 0)
data.drop([11004, 1150, 7829], inplace = True)



Next lets create some new features in our dataframe. The first being a total sum of all the spending for each customer titled Spending. We will also create another feature that includes the number of months each customer has been with the company. This will allow us to cluster the customers in groups of new and old as well as big and small spenders.



In [ ]:
data.columns

In [ ]:
data['Spending'] = data.MntWines + data.MntFruits + data.MntMeatProducts + data.MntFishProducts + data.MntSweetProducts + data.MntGoldProds
data['Time_With_Company'] = pd.to_datetime(data.Dt_Customer, dayfirst = True, format = '%d-%m-%Y')
data['Time_With_Company'] = pd.to_numeric(data.Time_With_Company.dt.date.apply(lambda z: (date.today() - z)).dt.days, downcast = 'integer') / 30

In [ ]:
data.head()

Lets now take a look at the Education feature. This will allow us to better understand the demographic of customer base.



In [ ]:
data.Education.unique()

There are only a few unique values for this feature so we can leave it be. Lets do the same with the Marital_Status feature now.



In [ ]:
data.Marital_Status.unique()

Seems like we have a larger number of unique values for this column. Some of these values are similar in definition so lets group them togheter to make our analysis easier.



In [ ]:
data.Marital_Status = data.Marital_Status.replace({"Divorced": "Single", "Together": "Partner","Married": "Partner", "Widow": "Single", "Alone": "Single", "Absurd": "Single","YOLO": "Single"})
#data.head()

Lets create some new features regarding the children of the customers



In [ ]:
data.head()

In [ ]:
data['Children'] = data.Kidhome + data.Teenhome
data['Has_Child'] = np.where(data.Children > 0, 'Has Child', 'No Child')
data.tail()

Finally, lets rename some of our column names.



In [ ]:
data= data.rename(columns = { "MntWines": "Wine",
                              "MntFruits": "Fruit",
                              "MntMeatProducts": "Meat",
                              "MntFishProducts": "Fish",
                              "MntSweetProducts" : "Sweets",
                              "MntGoldProds": "Gold"})
data.head()

In [ ]:
data =data.rename(columns = {"NumWebPurchases" : "Web",
                             "NumCatalogPurchases" : "Catalog",
                             "NumStorePurchases" : "Store",
                             "NumWebVisitsMonth" : "WebVisits"})


### Lets take a look at the values for web purchases



In [ ]:
data.Web.describe()

In [ ]:
#data.groupby('Web').count()
data.Web.value_counts()

Seems like we have some outlier values, lets remove them.



In [ ]:
outlier_IDs = data.loc[data.Web > 20].index
data.drop(outlier_IDs, inplace = True)
data.Web.value_counts()

We will do the same for the Catalog and Store columns



In [ ]:
#data.Catalog.unique()
data.Catalog.describe()


In [ ]:
x= data.loc[data.Catalog > 20].index
data.drop(x, inplace = True)

In [ ]:
data.Store.describe()
data.Store.value_counts()

Lets group together our clean dataframe and select the columns we wish to analyze.



In [ ]:
data= data[[ "Education", "Marital_Status", "Has_Child", "Children", "Age", "Income", "Spending", "Time_With_Company", "Wine", "Fruit", "Meat", "Sweets", "Gold", "Web", "Catalog", "Store", "WebVisits"]]

In [ ]:
data.head()

# Analysis

In [ ]:
import seaborn as sns
sns.countplot(data.Education).set(title = "Education level of Customer")
#sns.title = "Education level of Customer"

Seems like a majority of customers have a graduate degree (Bachelor's). It also seems like the second highest group includes customers with PhD's. Lets look at marital status now.



In [ ]:
sns.countplot(data.Marital_Status).set(title  = "Marital Status")

Seems like the number of customers in a relationship is almost double the number of single customers. This could be useful for marketing purposes. Let's move on to the child status of our customers.



In [ ]:
sns.countplot(data.Has_Child).set(title = "Child status of Customer")

An overwhelming majority of customers have at least one child. Now what about the age of our customers?



In [ ]:
sns.histplot(data.Age, color = 'midnightblue').set(title = "Ages of Customers", ylabel = None, xlabel = "Age")
print("The Average Age of our customer is", round(np.mean(data.Age)), 'year \n')

The age of our customers varies over a wide range. However it seems like most of our customers are middle aged (40-60 years).

